# Подсчет входящих людей

In [3]:
import cv2
import numpy as np

video_path = "people.mp4"
cam = cv2.VideoCapture(video_path)
_, background = cam.read()
background = cv2.cvtColor(background, cv2.COLOR_BGR2HSV)

people_count = 0
last_detection_time = 0
cooldown_time = 5 # кулдаун на время вхождения одного человека
frame_rate = cam.get(cv2.CAP_PROP_FPS)
frames_cooldown = int(frame_rate * cooldown_time)

while (True):
    success, frame = cam.read()
    
    if(success == False):
        cam.release()
        cam = cv2.VideoCapture(video_path)  
        continue
    
    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    background = cv2.addWeighted(background, 0.97, hsv_frame, 0.03, 0)

    diff = cv2.absdiff(hsv_frame, background)
    foreground_mask = cv2.inRange(diff, (18, 8, 23), (184, 205, 211))
    
    kernel = np.ones((11, 11), np.uint8)
    morph = cv2.morphologyEx(foreground_mask, cv2.MORPH_CLOSE, kernel)
    
    connectivity = 4 
    output = cv2.connectedComponentsWithStats(morph, connectivity, cv2.CV_32S)

    num_labels = output[0]
    labels = output[1]
    stats = output[2]
    
    filtered = np.zeros_like(morph)
    detected = False
    
    for i in range(1, num_labels):
        area = stats[i, cv2.CC_STAT_AREA]
        top = stats[i, cv2.CC_STAT_TOP]
        left = stats[i, cv2.CC_STAT_LEFT]
        width = stats[i, cv2.CC_STAT_WIDTH]
        height = stats[i, cv2.CC_STAT_HEIGHT]
        
        if (area >= 30000):
            detected = True
            filtered[np.where(labels == i)] = 255

            cv2.rectangle(frame, (left, top), (left + width, top + height), (0, 255, 0), 2)
            
    current_time = cam.get(cv2.CAP_PROP_POS_FRAMES)
    if detected and (current_time - last_detection_time > frames_cooldown):
        people_count += 1
        last_detection_time = current_time # После детекции человека срабатывает кулдаун
        
    cv2.putText(frame, f"Count: {people_count}", (100, 100), cv2.FONT_HERSHEY_TRIPLEX , 3, (0, 255, 0), 2)
    
    # cv2.imshow('frame', hsv_frame[:, :, 2])
    # cv2.imshow('back', background[:, :, 2])
    
    # cv2.imshow('diff0', diff[:, :, 0])
    # cv2.imshow('diff1', diff[:, :, 1])
    # cv2.imshow('diff2', diff[:, :, 2])
    
    # cv2.imshow('for_mask', foreground_mask)
    # cv2.imshow('morph', morph)
    # cv2.imshow('mask_filtered', filtered)
    
    cv2.imshow('frame', frame)
    
    key = cv2.waitKey(10) & 0xFF
    
    if (key == ord('q')):
        break

print(f"People total: {people_count}")
cam.release()
cv2.destroyAllWindows()
cv2.waitKey(10)

People total: 3


-1

# Я на фоне Breaking Bad

## Тут использовал медиапайп, чтобы нормально себя выделить на видео

In [5]:
import cv2
import mediapipe as mp
import numpy as np

mp_selfie_segmentation = mp.solutions.selfie_segmentation
segment = mp_selfie_segmentation.SelfieSegmentation(model_selection=1)

im = cv2.imread('breaking bad.jpg')
height, width, _ = im.shape

video_path = "me.mp4"
cam = cv2.VideoCapture(video_path)

while True:
    success, frame = cam.read()
    if not success:
        break
    
    frame_height, frame_width, _ = frame.shape
    scale = min(width / frame_width / 2, height / frame_height / 2)
    new_size = (int(frame_width * scale), int(frame_height * scale))
    frame = cv2.resize(frame, new_size)

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = segment.process(rgb_frame)
    mask = results.segmentation_mask
    
    mask = (mask > 0.6).astype(np.uint8) * 255
    mask = cv2.GaussianBlur(mask, (5, 5), 0)

    bg = np.zeros_like(im)

    x_offset = (width - new_size[0]) // 2
    y_offset = (height - new_size[1]) // 2

    bg[y_offset:y_offset + new_size[1], x_offset:x_offset + new_size[0]] = frame

    full_mask = np.zeros_like(im[:, :, 0])
    full_mask[y_offset:y_offset + new_size[1], x_offset:x_offset + new_size[0]] = mask

    mixed_clone = cv2.seamlessClone(bg, im, full_mask, (width // 2, height - 300), cv2.NORMAL_CLONE)
    
    cv2.imshow('mix', mixed_clone)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()
